# **Promedio de precios por Segmento**


In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre or 'df_junio_2023_queretaro' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [2]:
df_alfa_marzo_2024_queretaro.columns

Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Seguridad',
       'Colonia', 'CP', 'url'],
      dtype='object')

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    # Verificar si la columna 'm2_contruido' existe en el DataFrame
    #if 'm2_contruido' in df.columns:
        # Renombrar la columna
    df.rename(columns={'Categoria': 'categoria','Precio':'precio'}, inplace=True)
    # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria', 'precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


In [5]:
# Diccionario para almacenar el total de precios por categoría
total_precios_por_segmento = {}

# Diccionario para almacenar el número de observaciones por categoría
num_observaciones_por_segmento = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Añadir una nueva columna que extraiga el segmento
    #df['segmento'] = df['categoria'].str[0] # Letra
    df.loc[:, 'segmento'] = df['categoria'].str[0]

    # Calcular el total de precios por segmento en el DataFrame actual
    for segmento, group in df.groupby('segmento'):
        total_precios_por_segmento[segmento] = total_precios_por_segmento.get(segmento, 0) + group['precio'].sum()
        num_observaciones_por_segmento[segmento] = num_observaciones_por_segmento.get(segmento, 0) + len(group)

# Diccionario para almacenar el promedio de precios por segmento
promedio_precios_por_segmento = {}

# Calcular el promedio de precios por segmento
for segmento in total_precios_por_segmento:
    promedio_precios_por_segmento[segmento] = total_precios_por_segmento[segmento] / num_observaciones_por_segmento[segmento]

# Convertir el diccionario a un DataFrame
promedio_precios_por_segmento_df = pd.DataFrame(list(promedio_precios_por_segmento.items()), columns=['segmento', 'promedio_precio'])
promedio_precios_por_segmento_df


C:\Users\yoe11\AppData\Local\Temp\ipykernel_20888\4174865843.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_20888\4174865843.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_20888\4174865843.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,segmento,promedio_precio
0,A,3.561870e+06
1,B,2.812419e+06
2,C,2.078379e+06
3,D,1.380580e+06
4,E,1.971841e+07
5,L,1.640307e+07
6,S,6.185863e+06


In [6]:
# Definir el orden deseado para los segmentos
orden_segmentos = ['S', 'A', 'B', 'C', 'D', 'E']

# Convertir la columna 'segmento' a una categoría con el orden especificado
promedio_precios_por_segmento_df['segmento'] = pd.Categorical(promedio_precios_por_segmento_df['segmento'], 
                                                              categories=orden_segmentos, 
                                                              ordered=True)

# Ordenar el DataFrame según la columna 'segmento'
promedio_precios_por_segmento_df.sort_values('segmento', inplace=True)
promedio_precios_por_segmento_df

,segmento,promedio_precio
6,S,6.185863e+06
0,A,3.561870e+06
1,B,2.812419e+06
2,C,2.078379e+06
3,D,1.380580e+06
4,E,1.971841e+07
5,NaN,1.640307e+07


In [7]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure()
# Agregar las barras al gráfico
fig.add_trace(go.Bar(
    x=promedio_precios_por_segmento_df['segmento'],
    y=promedio_precios_por_segmento_df['promedio_precio'],
    text=promedio_precios_por_segmento_df['promedio_precio'],
    texttemplate='%{text:.2s}',  # Mostrar los valores en las barras
    textposition='inside',
    marker_color=colores,  # Usar la paleta de colores definida
))
# Actualizar el diseño de la gráfica
fig.update_layout(
    #title='Promedio de precios por segmento',
    xaxis_title='Segmento',
    yaxis=dict(
        title='Promedio de Precio',
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title='Segmento',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    margin=dict(l=10, r=10, t=50, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_segmento', carpeta='assets/graficas')
fig.show()